# my first gpt model

In [10]:
# starting with a data set to train on
!wget https://https://www.o-bible.com/download/kjv.txt 

zsh:1: command not found: wget


In [11]:
# read it and inspect it
with open('kjv.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    

In [12]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  4404381


In [13]:
# let's look at the first 1000 characters
print(text[:1000])

Ge1:1 In the beginning God created the heaven and the earth.
Ge1:2 And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.
Ge1:3 And God said, Let there be light: and there was light.
Ge1:4 And God saw the light, that it was good: and God divided the light from the darkness.
Ge1:5 And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.
Ge1:6 And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters.
Ge1:7 And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so.
Ge1:8 And God called the firmament Heaven. And the evening and the morning were the second day.
Ge1:9 And God said, Let the waters under the heaven be gathered together unto one place, and let the dry land appear: and it was so.
Ge1:10 And God c

In [17]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz
73


In [20]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hello world"))
print(decode(encode("hello world")))

[54, 51, 58, 58, 61, 1, 69, 61, 64, 58, 50]
hello world
